In [1]:
import os
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.font_manager as fm
import seaborn as sns

In [2]:
import pickle
import pandas as pd
import numpy as np

with open(r"/cluster/home/vjimenez/adv_pa_new/results/nlp/sentiment_analysis/levenshtein.pkl", 'rb') as f:
    data_lev = pickle.load(f)
df = pd.DataFrame(data_lev)
df["method"] = "levenshtein"

with open(r"/cluster/home/vjimenez/adv_pa_new/results/nlp/sentiment_analysis/adversarial.pkl", 'rb') as f:
    data_adv = pickle.load(f)
df2 = pd.DataFrame(data_adv)
df2["method"] = "adversarial"

with open(r"/cluster/home/vjimenez/adv_pa_new/results/nlp/sentiment_analysis/adversarial-inverted.pkl", 'rb') as f:
    data_advinv = pickle.load(f)
df3 = pd.DataFrame(data_advinv)
df3["method"] = "adversarial_inverted"

df = pd.concat([df, df2, df3])

In [7]:
COLORS_DICT = {
    "levenshtein": "tab:blue",
    "adversarial": "tab:red",
    "adversarial_inverted": "limegreen"
}

METRICS_DICT = {
    "AFR_true": r"AFR$_{\operatorname{T}}$",
    "logPA": "PA"
}

In [4]:
df["intensity_plot"] = np.log2(df["intensity"])

dfplot = df.loc[(df["intensity_plot"] < 9)]

In [12]:
from matplotlib.ticker import ScalarFormatter
metric = "AFR_true" #"logPA"

fontname = "DejaVu Serif"
_ = fm.findfont(fm.FontProperties(family=fontname))

# Create a line plot for PGD attack type with Seaborn
_, ax = plt.subplots(figsize=(2 * 3.861, 2 * 2.7291))
fontsize = 18 
sns.set(font_scale=1.9)
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = fontname
sns.set_style("ticks")

sns.lineplot(
    data=dfplot,
    ax=ax,
    x="intensity_plot",
    y=metric,
    hue="method",
    style="method",
    palette=COLORS_DICT,
    dashes=False,
    marker="o",
    linewidth=3,
    legend=False
)

# Set the title with the desired font name and font size

ax.minorticks_on()
ax.set_xticks([i for i in range(9)])
ax.set_xlabel("Shift Power", fontname=fontname)
ax.set_title("IMDB classification", fontname=fontname)

ax.tick_params(axis="both", which="both", direction="in")
xticks_font = fm.FontProperties(family=fontname)
for tick in ax.get_xticklabels():
    tick.set_fontproperties(xticks_font)
ax.grid(linestyle="--")

if metric == "logPA":
    ax.set_yticks([0, -5000, -10000, -15000])
    ax.set_yticklabels(["0", r"$-0.5$", r"$-1.0$", r"$-1.5$"])
    ax.text(0, 1.03, r'$\times 10^4$', transform=ax.transAxes, fontname=fontname, fontsize=18, verticalalignment='center', horizontalalignment='left')
    # ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=True))
    # ax.yaxis.get_offset_text().set_visible(True)  # Show the offset text (scale factor)
    # ax.yaxis.offsetText.set_text(r"$\times 10^{-4}$")  # Set custom scale factor label

ax.set_ylabel(
    METRICS_DICT[metric],
    fontname=fontname
)
# ax.set_yscale("linear") 

handles = [mlines.Line2D([], [], color='tab:blue', linewidth=3), mlines.Line2D([], [], color='limegreen', linewidth=3), mlines.Line2D([], [], color='tab:red', linewidth=3)]
labels = ["Levenshtein", "Amplification", "Contradiction"]
ax.legend(
    handles,
    labels,
    # loc="upper right",
    loc="lower left",
    # fontsize=12,
    handlelength=0.5,
    prop={
        "family": fontname,
        'size': 18
    }  
)


plt.tight_layout()
fname = osp.join(
    r"/cluster/home/vjimenez/adv_pa_new/results/nlp", 
    f"presentation_afrt.pdf"
)
plt.savefig(fname, dpi=300)
plt.clf()
plt.show()


<Figure size 772.2x545.82 with 0 Axes>